In [1]:
# Importing necessary libraries
from pythtb import *
import matplotlib.pyplot as plt

# Initializing the Wannier90 model by reading the data files
pgda=w90(r"Ham2",r"wannier90")
my_model=pgda.model(min_hopping_norm=0.01) # only hoppings with norm greater than 0.01 are considered

# Task 1

In [ ]:
# Choosing the path
path=[[0.5,0.0,0.0],[0.5,-0.5,0.0],[0.0,0.0, 0.0],[0.5,0.5,0.5],[0.5,0.0,0.0]]
k_label=(r'$X$', r'$M$', r'$\Gamma$', r'$R$', r'$X$')
(k_vec,k_dist,k_node)=my_model.k_path(path,1001,report=False)

# Finding the energy eigenvalues at all k-points
evals=my_model.solve_all(k_vec)
energies = evals - 6.87 # Subtructing the Fermi energy

# Plotting the bands
fig, ax = plt.subplots()
for i in range(evals.shape[0]):
    ax.plot(k_dist,energies[i],"k-")
for n in range(len(k_node)):
    ax.axvline(x=k_node[n],linewidth=0.5, color='k')
ax.set_xlabel("Path in k-space")
ax.set_ylabel("Band energy (eV)")
ax.set_xlim(k_dist[0],k_dist[-1])
ax.set_ylim(-3, 3)
ax.set_xticks(k_node)
ax.set_xticklabels(k_label)
fig.tight_layout()

plt.show()

# Task 2

In [2]:
# To write the band velocity operator at a k-point
def calculate_v(k_vec, dk=0.01):
    H = my_model._gen_ham([k_vec[0], k_vec[1], k_vec[2]])
    Hx = my_model._gen_ham([k_vec[0]+dk, k_vec[1], k_vec[2]])
    Hy = my_model._gen_ham([k_vec[0], k_vec[1]+dk, k_vec[2]])
    Hz = my_model._gen_ham([k_vec[0], k_vec[1], k_vec[2]+dk])

    vx = (Hx-H)/dk
    vy = (Hy-H)/dk
    vz = (Hz-H)/dk
    return np.array((vx, vy, vz))

In [5]:
# To get energy eigenvalues and corresponding OAMs at a k-point
def solve_and_calculate_L(k_vec, eta=0.01):
    v = calculate_v(k_vec)                  # velocity using forward finite difference
    #v = my_model.derivative_ham(k_vec)     # velocity using exact derivative formula
    
    # Getting energy eigenvalues and vectors
    (eig_values, eig_vectors) = my_model.solve_one(k_vec, eig_vectors=True)
    
    # Calculating 3 components of OAM
    Lx = np.zeros(eig_values.shape[0])
    for n in range(eig_values.shape[0]):
        temp = 0
        for m in range(eig_values.shape[0]):
            if m==n:
                continue
            temp += (eig_vectors[n].conjugate().dot(v[1]).dot(eig_vectors[m])*eig_vectors[m].conjugate().dot(v[2]).dot(eig_vectors[n]))/(eig_values[n]-eig_values[m]+complex(0,eta))
            temp -= (eig_vectors[n].conjugate().dot(v[2]).dot(eig_vectors[m])*eig_vectors[m].conjugate().dot(v[1]).dot(eig_vectors[n]))/(eig_values[n]-eig_values[m]+complex(0,eta))
        Lx[n] = np.imag(temp)
    Ly = np.zeros(eig_values.shape[0])
    for n in range(eig_values.shape[0]):
        temp = 0
        for m in range(eig_values.shape[0]):
            if m==n:
                continue
            temp += (eig_vectors[n].conjugate().dot(v[2]).dot(eig_vectors[m])*eig_vectors[m].conjugate().dot(v[0]).dot(eig_vectors[n]))/(eig_values[n]-eig_values[m]+complex(0,eta))
            temp -= (eig_vectors[n].conjugate().dot(v[0]).dot(eig_vectors[m])*eig_vectors[m].conjugate().dot(v[2]).dot(eig_vectors[n]))/(eig_values[n]-eig_values[m]+complex(0,eta))
        Ly[n] = np.imag(temp)
    Lz = np.zeros(eig_values.shape[0])
    for n in range(eig_values.shape[0]):
        temp = 0
        for m in range(eig_values.shape[0]):
            if m==n:
                continue
            temp += (eig_vectors[n].conjugate().dot(v[0]).dot(eig_vectors[m])*eig_vectors[m].conjugate().dot(v[1]).dot(eig_vectors[n]))/(eig_values[n]-eig_values[m]+complex(0,eta))
            temp -= (eig_vectors[n].conjugate().dot(v[1]).dot(eig_vectors[m])*eig_vectors[m].conjugate().dot(v[0]).dot(eig_vectors[n]))/(eig_values[n]-eig_values[m]+complex(0,eta))
        Lz[n] = np.imag(temp)
    return (eig_values, np.array((Lx, Ly, Lz)))

In [ ]:
# Choosing the path
path=[[-0.5,-0.5,-0.5],[0.0,0.0, 0.0],[0.5,0.5,0.5]]
k_label=(r'$R$', r'$\Gamma$', r'$R$')
(k_vec,k_dist,k_node)=my_model.k_path(path,501,report=False)

# Calculating OAM
evals = np.zeros((64, k_vec.shape[0]))
L = np.zeros((3, 64, k_vec.shape[0]))
for i in range(k_vec.shape[0]):
    (evals[:, i], L[:,:, i]) = solve_and_calculate_L(k_vec[i])
    if i%50 == 0:
        print(i+1, "points done.")

energies = evals-6.87
colors = L[0]+L[1]+L[2]     # L_111

In [ ]:
# Plotting
plt.set_cmap('bwr')
cmax = 30
for i in range(evals.shape[0]):
    plt.scatter(k_dist, energies[i], s=1, c=colors[i])
    plt.clim(-cmax, cmax)
plt.colorbar(label="$L_{111}$")
for n in range(len(k_node)):
    plt.axvline(x=k_node[n],linewidth=0.5, color='k')
plt.xlabel("Path in k-space")
plt.ylabel("Band energy (eV)")
plt.xlim(k_dist[0],k_dist[-1])
plt.ylim(-2.5, 2.5)
plt.xticks(k_node, labels=k_label)
plt.show()

# Task 3


$j^z_x=\frac{1}{2} \left\{ v_x,L_z \right\}$

$\Omega^{z}_{n,xy}=-2 Im \sum_{m \neq n}\frac{ \langle n(k)|j^{z}_{x}|m(k)\rangle \langle m(k)|v_{y}|n(k)\rangle}{( E_n(k)-E_m(k))^2}$

In [8]:
# To get energy eigenvalues and corresponding OBC (Omega^z_xy) at a k-point
def solve_and_calculate_Omega_xyz(k_vec, eta=0.0001):
    v = calculate_v(k_vec)                  # velocity using forward finite difference
    #v = my_model.derivative_ham(k_vec)     # velocity using exact derivative formula
    (eig_values, eig_vectors) = my_model.solve_one(k_vec, eig_vectors=True)
    
    # Calculating L_z
    Lz = np.zeros(eig_values.shape[0])
    for n in range(eig_values.shape[0]):
        temp = 0
        for m in range(eig_values.shape[0]):
            if m==n:
                continue
            temp += (eig_vectors[n].conjugate().dot(v[0]).dot(eig_vectors[m])*eig_vectors[m].conjugate().dot(v[1]).dot(eig_vectors[n]))/(eig_values[n]-eig_values[m]+complex(0,eta))
            temp -= (eig_vectors[n].conjugate().dot(v[1]).dot(eig_vectors[m])*eig_vectors[m].conjugate().dot(v[0]).dot(eig_vectors[n]))/(eig_values[n]-eig_values[m]+complex(0,eta))
        Lz[n] = np.imag(temp)
    
    # Calculating Omega^z_xy
    Omega_xyz = np.zeros(eig_values.shape[0])
    for n in range(eig_values.shape[0]):
        temp = 0
        for m in range(eig_values.shape[0]):
            if m==n:
                continue
            temp -= (Lz[m]+Lz[n])*(eig_vectors[n].conjugate().dot(v[0]).dot(eig_vectors[m])*eig_vectors[m].conjugate().dot(v[1]).dot(eig_vectors[n]))/(eig_values[n]-eig_values[m]+complex(0,eta))**2
        Omega_xyz[n] = np.imag(temp/2)
    return (eig_values, Omega_xyz)

In [ ]:
# Choosing the path
path=[[0.0,0.0, 0.0],[0.5,0.0, 0.0],[0.5,0.5,0.0],[0.0,0.0, 0.0],[0.5,0.5,0.5],[0.5,0.0, 0.0]]
k_label=(r'$\Gamma$', r'$X$', r'$M$', r'$\Gamma$', r'$R$', r'$X$')
(k_vec,k_dist,k_node)=my_model.k_path(path,501,report=False)

# Calculating OBC
evals = np.zeros((64, k_vec.shape[0]))
Omega_xyz = np.zeros((64, k_vec.shape[0]))
for i in range(k_vec.shape[0]):
    (evals[:, i], Omega_xyz[:, i]) = solve_and_calculate_Omega_xyz(k_vec[i])
    if i%100 == 0:
        print(i+1, "points done.")

colors = Omega_xyz
energies = evals -6.87

In [ ]:
# Plotting
cmax = 0.01
plt.set_cmap('bwr')
for i in range(energies.shape[0]):
    plt.scatter(k_dist,energies[i], s=1, c=Omega_xyz[i])
    plt.clim(-cmax, cmax)
plt.colorbar(label="$\\Omega^z_{xy}$")
for n in range(len(k_node)):
    plt.axvline(x=k_node[n],linewidth=0.5, color='k')
plt.xlabel("Path in k-space")
plt.ylabel("Band energy (eV)")
plt.xlim(k_dist[0],k_dist[-1])
plt.ylim(-1.25, 1.25)
plt.xticks(k_node, labels=k_label)
plt.show()

# Supplementary

In [ ]:
# Choosing the path
path=[[-0.5,-0.5,-0.0],[0.0,0.0, 0.0],[0.5,0.5,0.0]]
k_label=(r'$M$', r'$\Gamma$', r'$M$')
(k_vec,k_dist,k_node)=my_model.k_path(path,501,report=False)

# Calculating OAM
evals = np.zeros((64, k_vec.shape[0]))
L = np.zeros((3, 64, k_vec.shape[0]))
for i in range(k_vec.shape[0]):
    (evals[:, i], L[:,:, i]) = solve_and_calculate_L(k_vec[i])
    if i%100 == 0:
        print(i+1, "points done.")

energies = evals-6.87
colors = L[0] + L[1]

In [ ]:
# Plotting
plt.set_cmap('bwr')
cmax = 30
for i in range(evals.shape[0]):
    plt.scatter(k_dist, energies[i], s=1, c=colors[i])
    plt.clim(-cmax, cmax)
plt.colorbar(label="$L_{110}$")
for n in range(len(k_node)):
    plt.axvline(x=k_node[n],linewidth=0.5, color='k')
plt.xlabel("Path in k-space")
plt.ylabel("Band energy (eV)")
plt.xlim(k_dist[0],k_dist[-1])
plt.ylim(-2.5, 2.5)
plt.xticks(k_node, labels=k_label)
plt.show()